In [ ]:
import requests
import pandas as pd
from IPython.display import display
import re

API_KEY = "01946b8515c545443cdcd262a884a88dab1be54962aad37f4f93c3420cc49844"
headers = {"X-API-Key": API_KEY}

def extract_data (urls): 

    params = {
        "limit": 1000,
        "page": 1
    }

    all_results = []

    while True:
        response = requests.get(url, headers=headers, params=params)
        if response.status_code != 200:
            print("Error:", response.status_code, response.text)
            break

        data = response.json()
        results = data.get('results', [])
        all_results.extend(results)

        if len(results) < params['limit']:
            break

        params['page'] += 1

    print(f"Total records retrieved: {len(all_results)}")
    print(f'extract_output {all_results[:3]}') 


# Getting worldwide sensor locations and unpack the country
url= "https://api.openaq.org/v3/locations"
extract_data(url)
df_locations = pd.DataFrame(all_results)
print(df_locations.head(3))
df_country = df_locations['country'].apply(pd.Series)
df_country.columns= ['country_id', 'country_code', 'country_name']
df_locations=pd.concat([df_locations,df_country],axis=1)
df_locations.drop(['owner', 'provider', 'isMobile', 'isMonitor', 'instruments', 'licenses', 'bounds', 'distance', 'country'], axis=1, inplace=True)
df_locations.rename(columns={'id': 'location_id', 'name': 'location_name'}, inplace=True)
print(f'locations dataframe output : {df_locations[:3]}')
df_locations.to_csv('pollution_data/locations_clean.csv')

# get the sensor ID list  along with the location id

location_sensor_pairs = [(location["id"], sensor["id"]) 
                         for location in all_results 
                         for sensor in location["sensors"]]
print(f'locations_sensors_pairs_list output :  {location_sensor_pairs[:5]}')

# generate URLs list
urls_list = [f"https://api.openaq.org/v3/sensors/{pair[1]}" for pair in location_sensor_pairs]
print(f'url_list {urls_list[:3]}')

# generate sensor data  dataframe
final_results = []
for url in urls_list : 
    extract_data(url)
    final_results.extend(all_results)
    final_df = pd.DataFrame(final_results)
    print (f'final df output:' {final_df[:3]})


# generate the location id list and dataframe 
sensor_id_list= [location[0] for location in  location_sensor_pairs]
print(sensor_id_list[:3])
sensor_id_df = pd.DataFrame(sensor_id_list)
print(f'sensor_id_df {sensor_id_df.head(3)}')

Total records retrieved: 19465
extract_output [{'id': 3, 'name': 'NMA - Nima', 'locality': None, 'timezone': 'Africa/Accra', 'country': {'id': 152, 'code': 'GH', 'name': 'Ghana'}, 'owner': {'id': 4, 'name': 'Unknown Governmental Organization'}, 'provider': {'id': 209, 'name': 'Dr. Raphael E. Arku and Colleagues'}, 'isMobile': False, 'isMonitor': True, 'instruments': [{'id': 2, 'name': 'Government Monitor'}], 'sensors': [{'id': 6, 'name': 'pm10 µg/m³', 'parameter': {'id': 1, 'name': 'pm10', 'units': 'µg/m³', 'displayName': 'PM10'}}, {'id': 5, 'name': 'pm25 µg/m³', 'parameter': {'id': 2, 'name': 'pm25', 'units': 'µg/m³', 'displayName': 'PM2.5'}}], 'coordinates': {'latitude': 5.58389, 'longitude': -0.19968}, 'licenses': None, 'bounds': [-0.19968, 5.58389, -0.19968, 5.58389], 'distance': None, 'datetimeFirst': None, 'datetimeLast': None}, {'id': 4, 'name': 'NMT - Nima', 'locality': None, 'timezone': 'Africa/Accra', 'country': {'id': 152, 'code': 'GH', 'name': 'Ghana'}, 'owner': {'id': 4, '

In [ ]:
# generate the sensor data fataframe



[3, 3, 4]
   0
0  3
1  3
2  4


In [ ]:
df.to_csv("pollution_data/locations_bbox.csv")

In [4]:


# Sample list of locations with sensors
locations_data = [ 
    {"id": 3, "sensors": [{"id": 6}, {"id": 5}]}, 
    {"id": 4, "sensors": [{"id": 7}, {"id": 8}]}, 
    {"id": 5, "sensors": [{"id": 10}, {"id": 9}]} 
]

# List comprehension to extract (location_id, sensor_id) pairs
location_sensor_pairs = [(location["id"], sensor["id"]) 
                         for location in locations_data 
                         for sensor in location["sensors"]]

# Print extracted pairs
print(location_sensor_pairs)


--------


import pandas as pd
import json

# Sample JSON data (as a list of dictionaries)
json_data = [
    {
        "location_id": 101,
        "sensors": [
            {"id": 6, "name": "pm10 Âµg/mÂ³", "parameter": {"id": 1, "name": "pm10", "units": "Âµg/mÂ³", "displayName": "PM10"}},
            {"id": 5, "name": "pm25 Âµg/mÂ³", "parameter": {"id": 2, "name": "pm25", "units": "Âµg/mÂ³", "displayName": "PM2.5"}}
        ]
    },
    {
        "location_id": 102,
        "sensors": [
            {"id": 8, "name": "co ppm", "parameter": {"id": 3, "name": "co", "units": "ppm", "displayName": "CO"}},
            {"id": 7, "name": "no2 ppb", "parameter": {"id": 4, "name": "no2", "units": "ppb", "displayName": "NO2"}}
        ]
    }
]

# Convert JSON to DataFrame
df_locations = pd.DataFrame(json_data)

# Explode 'sensors' list
df_exploded = df_locations.explode('sensors')

# Extract 'id' from sensors dictionary
df_exploded['sensor_id'] = df_exploded['sensors'].apply(lambda x: x['id'])

# Keep only relevant columns
df_result = df_exploded[['location_id', 'sensor_id']]

print(df_result)

SyntaxError: invalid syntax (3568947690.py, line 17)

In [ ]:
import json

# Sample JSON data
json_data = [
    {
        "location_id": 101,
        "sensors": [
            {"id": 6, "name": "pm10 Âµg/mÂ³"},
            {"id": 5, "name": "pm25 Âµg/mÂ³"}
        ]
    },
    {
        "location_id": 102,
        "sensors": [
            {"id": 8, "name": "co ppm"},
            {"id": 7, "name": "no2 ppb"}
        ]
    }
]

# Extracting location_id and sensor_id
sensor_list = [(entry["location_id"], sensor["id"]) for entry in json_data for sensor in entry["sensors"]]

# Print the result as a list of tuples
print(sensor_list)

In [ ]:



# Extraire le numéro de sensor depuis l'URL
#sensor_id_match = re.search(r'/sensors/(\d+)', url)
#sensor_id = int(sensor_id_match.group(1)) if sensor_id_match else None

# Ajouter la colonne 'sensor_id' au DataFrame
#df_sensor['sensor_id'] = sensor_id

# Afficher le DataFrame
#display(df_sensor)



#url = "https://api.openaq.org/v3/sensors/3917/days/monthly"
#url_1 = "https://api.openaq.org/v3/sensors/3917"
#url_2 = "https://api.openaq.org/v3/sensors/3917/measurements"
#url = "https://api.openaq.org/v3/locations?bbox=-10.5,36,9.6,51.1"
#url = "https://api.openaq.org/v3/sensors/3917/days/monthly"
#url = "https://api.openaq.org/v3/sensors/3917/years"
#url = "https://api.openaq.org/v3/sensors/3917/days"
#url = "https://api.openaq.org/v3/sensors/3917/measurements"
#url = "https://api.openaq.org/v3/parameters"
#url = "https://api.openaq.org/v3/locations"
#url = "https://api.openaq.org/v3/countries"
